# Exemplo de Reprodução dos Resultados de Classificação de Uso e Cobertura da Terra utilizando o pacote **sits.rep**

A reprodutibilidade das pesquisas científicas tem sido um tópico de grande discussão no meio acadêmico. A sua importância é discutida em todas as ciências, dado que é um dos pilares do método científico, com o objetivo de garantir a verificação e confiabilidade da metodologia e dos resultados produzidos em uma pesquisa. Apesar de sua importância, diversos trabalhos das mais variadas áreas da ciência têm apontado dificuldades em reproduzir trabalhos científicos, revelando uma possível crise na reprodutibilidade. Na área de economia, por exemplo, o trabalho de Gertler concluiu que, dos 203 artigos empíricos analisados, somente 14\% foram possíveis de replicação. Um dos problemas apontado por Gertler foi a falta de disponibilização aberta dos códigos e dados para reproduzir os resultados. 

Essa dificuldade enfrentada pelos pesquisadores tem motivado os principais periódicos a elaborar documentos de boas práticas e diretrizes que ajudem os pesquisadores a organizar e compartilhar dados, códigos e artefatos de suas publicações. As revistas Science e Nature, por exemplo, elaboraram regras para que novas publicações usem formas padronizadas e abertas para o acesso, armazenamento e citação tanto dos dados quanto dos métodos utilizados. Adicionalmente, diversos grupos de pesquisa têm proposto e desenvolvido novas ferramentas computacionais para auxiliar os pesquisadores nas questões de reprodutibilidade de seus trabalhos. Essas ferramentas compreendem desde sistemas gerais de catalogação de metadados das pesquisas até ferramentas que buscam a reprodução automática das análises de dados. Nesse último caso, essas ferramentas são voltadas para lidar com problemas específicos de uma área da ciência, já que os tipos de dados e métodos computacionais variam de ciência para ciência, dificultando desenvolver uma solução genérica. 

Na área de observação da Terra, esse assunto tem criado notoriedade recentemente com projetos e conferências obrigando os pesquisadores a disponibilizarem os dados e códigos em repositórios abertos. Entretando, segundo Alber, existe uma grande dificuldade em realizar o processo de reprodutibilidade em trabalhos que envolvem análise de dados de Observação da Terra. Isso ocorre por causa do grande volume de dados que são gerados todos os dias, o que torna o processamento dos dados demorado e limita o espaço de armazenamento. 

Dependendo dos métodos computacionais utilizados para efetuar análise de dados, é exigido um grande esforço por parte dos cientistas para garantir a reprodutibilidade dos resultados, já que repetir uma análise com os mesmos dados de entrada e métodos computacionais não garante aos cientistas obter os mesmos resultados. Diversos fatores podem influenciar no resultado final, como falta de informações sobre o pré-processamento dos dados, versões diferentes dos pacotes de dependência, execução em um sistema operacional diferente e o uso da semente, utilizada para a geração de números aleatórios,  diferente em cada execução. Por tanto, é de grande importância armazenar todo o contexto da execução além do resultado final e criar métodos para que, através desse contexto, os pesquisadores possam reproduzir os resultados originais obtidos em uma análise de dados.

A questão da reprodutibilidade se torna ainda mais relevante em projetos de pesquisas envolvendo grupos multidisciplinares e colaborativos, sendo necessário 
registrar todo o processo executado por cada membro envolvido no desenvolvimento dos resultados de uma pesquisa. Com base nisso, uma ferramenta para fins 
específicos e bem definidos pode acelerar e facilitar o processo de reprodutibilidade ao coletar todas as informações necessárias durante a execução das etapas
de uma determinada pesquisa. Isto facilita a publicação de documentos científicos reprodutíveis, conforme exigido atualmente por diferentes periódicos científicos.

Esse trabalho parte da hipótese de que análises com dados de Observação da Terra são passíveis de reprodução. Partimos do pressuposto que é necessário armazenar dados de contexto referentes aos experimentos intermediários dentro da análise da dados. Com esse contexto, iremos simular o ambiente original para obter os mesmos resultados. Como estudo de caso, iremos focar em reproduzir os resultados em uma ferramenta específica para análise de dados de Observação da Terra, que é o Satellite Image Time Series (sits). 

# Metodologia

O sits.rep é a materialização da hipótese de que os resultados obtido através de ferramentas de análises dados de Observação da Terra são passivéis de reprodução. Como foi observado nos trabalhos que buscam tratar a reprodutibilidade, há sempre um foco em uma determinada linguagem de programação, formatos de dados e como os resultados serão obtidos. Essa restrição acontece devido os diferentes algoritmos e dados que cada pesquisa exige. Por tanto, sits.rep segue esse príncipio focando na ferramenta sits.

A metodologia proposta pelo sits.rep é reproduzir classificações realizadas com o pacote sits. Está metodologia está dividida em duas etapas: [Criação dos Experimentos](#Criação-dos-Experimentos), onde são guardadas informações de reprodutibilidade durante o processo de classificação de uso da terra, e [Reprodução dos Resultados](#Reprodução-dos-Resultados), onde um ambiente é criado para permitir a reprodução das classificações geradas. Ambas etapas são descritas a seguir.

## Criação dos Experimentos

Essa etapa o pesquisador desenvolve um script que será executado pelo sits.rep para armazenar os resultados. [Neste exemplo](./script-classificacao.ipynb), é explicado quais são as principais partes que um script de classificação deve conter para ser reproduzido. Durante a execução, é coletado o seu contexto que inclui parâmetros de entrada das funções, a semente do gerador de números aleatórios, o modelo treinado, o código original, resultados, informações de sistema operacional, dependências e o ambiente de programação.

Todos os dados de entrada usado pelos pesquisadores para treinar um modelo e classificar séries temporais, possuem versionamentos e estruturas de dados bem definidas pelos pacotes inSitu e EOCubes. Desde que esses dados se encontram hospedados em serviços web e são explicitamente versionados, é possível recuperar exatamente a mesma versão dos dados usadas no treinamento e classificações. Por tanto, não sendo necessário armazená-los para garantir a reprodutibilidade, bastando coletar as versões de ambos os pacotes.

O script que iremos utilizar a partir de agora se encontra no caminho a seguir:

In [1]:
script <- paste0(getwd(), "/classification.R")
script

[1] "/home/rafael/workspace/sits.rep/jupyter/classification.R"

Esse script possuí as mesmas funções do [exemplo](./script-classificacao.ipynb). Inicialmente, é necessário instalar o sits.rep caso não tenha:

In [ ]:
devtools::install_github('https://github.com/RafaMariano/sits.rep')

Depois, basta importar para a memória do R:

In [6]:
library(sits.rep)

O sits.rep disponibiliza duas funções, sendo uma para executar as classificações e outra para reproduzir. Focaremos nesse momento na função de classificação.

### Executando a Classificação com o sits.rep

Toda execução realizada pelo sits.rep é chamado de experimento. Para cada execução dos experimentos, será criado uma relação de árvore de dependências. Por padrão, a classificação sempre é raiz de uma árvore e única, logo, não irá existir duas classificações na mesma árvore. Futuramente será incluido a possibilidade de tratar o pós processamento, e nesse caso, esse experimento irá necessáriamente depender de um processo pai criado anteriormente, seja uma classificação ou outro experimento de pós processamento. Dessa forma, o sits.rep irá automaticamente incluir como dado de entrada no pós-processamento, o resultado do experimento pai.

A função para classificação do sits.rep (chamado de 'sits.rep_classify') recebe dois parâmetros de entrada: um nome único para árvore e um script de classificação. Como resultado dessa função, será criado um diretório com o nome da árvore e, dentro desse diretório, conterá experimentos de classificação e, futuramente, de pós-processamento. A imagem abaixo exemplifica essa estrutura de diretório. A árvore 'arv_1' possuí três experimentos: uma classificação e dois de pós-processamento (pos_bayes, mosaico). A ligação de parentesco entre os experimentos é dado pelo arquivo 'grafo.txt'. Dentro de cada experimento deverá conter uma cópia do script, o resultado da execução, um hash dos resultados, arquivo de contexto e outros dados particulares de cada execução. O arquivo de hash dos resultados será usado na fase de [Reprodução dos Resultados](#Reprodução-dos-Resultados) para que, após reproduzir o experimento, verificar se os resultados são iguais. O diretório reprodutível será abordado na seção [Reprodução dos Resultados](#Reprodução-dos-Resultados).

![alt text](img/arv_dep.png "Title")


Logo abaixo, é executado a função para classificação. O primeiro argumento é o nome da árvore que irá se chamar 'arv_1' e o segundo é o script.

In [4]:
sits.rep_classify("arv_1", script)

sits - satellite image time series analysis.
Loaded sits v1.12.6. See ?sits for help, citation("sits") for use in publication.
                See demo(package = "sits") for examples.
Created logger for sits package - DEBUG level at /tmp/RtmpXSk1RI/sits_debug1ecb65627109.log
Created logger for sits package - ERROR level at /tmp/RtmpXSk1RI/sits_error1ecb3155d1d1.log
Registered S3 method overwritten by 'xts':
  method     from
  as.zoo.xts zoo 
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
Registered S3 methods overwritten by 'forecast':
  method             from    
  fitted.fracdiff    fracdiff
  residuals.fracdiff fracdiff
Loaded inSitu e-sensing data sets v1.1.1. Use citation("inSitu") for use in publication.
Set session seed to 42 (disabled GPU, CPU parallelism)
Starting classification at 2019-11-27 14:28:20
Classification finished at 2019-11-27 14:29:02. Total elapsed time: 0.7 minute(s).
Starting classification at 2019-11-27 14:2

Finalizado a execução, é possível visualizar a árvore e o experimento armazenados no diretório padrão '~/sits.rep'.

In [9]:
list.dirs('~/sits.rep', recursive = TRUE)

[1] "/home/rafael/sits.rep"                                       
 [2] "/home/rafael/sits.rep/arv_1"                                 
 [3] "/home/rafael/sits.rep/arv_1/classification"                  
 [4] "/home/rafael/sits.rep/arv_1/classification/Classification"   
 [5] "/home/rafael/sits.rep/arv_1/classification/Classification/MT"
 [6] "/home/rafael/sits.rep/arv_1/classification/coverage"         
 [7] "/home/rafael/sits.rep/arv_1/classification/coverage/geom"    
 [8] "/home/rafael/sits.rep/arv_1/classification/result"           
 [9] "/home/rafael/sits.rep/arv_1/classification/result/raster"    
[10] "/home/rafael/sits.rep/arv_1/classification/result/rds"       
[11] "/home/rafael/sits.rep/arv_1/classification/train"

# Capturando dados de Contexto


Os dados de contexto são capturados através de sobrescrita, em tempo de execução, das principais funções presentes nos pacotes sits e base do R. Quando um script é executado pelo 'sits.rep_classify', devido às funções sobrescritas serem inicialmente carregadas na memória global do R, as funções do sits.rep serão utilizadas em vez das funções originais do sits. As funções sobrescritas capturam os parâmetros de entrada, armazenam em um repositório, e no final usam as respectivas funções originais, passando os mesmos parâmetros de entrada, para que o seu resultado seja retornado e o processamento prossiga como se a sobrecarga não existisse.

Para exemplificar essa idéia, logo abaixo iremos sobrescrever a função 'sits_coverage' do pacote sits. Essa função recebe quatro argumentos de entrada: 

In [45]:
sits_coverage(service = "EOCUBES",
              name = "MOD13Q1/006",
              bands = "ndvi",
              geom = sf::read_sf(paste0(getwd(), "/geom/geom.shp")))

r_objs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Para sobrescrever o 'sits_coverage', é necessário criar uma função com a mesma assinatura, ou seja, com o mesmo nome da função e quantidade de argumentos. No interior da função sobrescrita, é possível analisar os valores de cada parâmetro para tomar alguma decisão, por exemplo, armazenar esses parâmetros. Neste exemplo, o valor de cada parâmetro será impresso no terminal. O seu retorno deve ser o resultado da função original para que o processamento pode ser seguido como se a sobrecarga não existisse. Assim, é chamado a função original (acessado através do nome do pacote seguido por '::') para que o seu resultado seja o retorno da função sobrescrita. 

Logo abaixo é exemplificado essa idéia sobrescrevendo o 'sits_coverage' com o propósito de imprimir no terminal o valor armazenado por cada argumento da função e no final é retornado o resultado da função original presente no sits:

In [48]:
sits_coverage <- function(service, name, bands, geom){
    print(service)
    print(name)
    print(bands)
    print(geom)
    
    return(sits::sits_coverage(service = service, name = name, bands = bands, geom = geom)) 
}

Agora, ao executar a função 'sits_coverage' é impresso impresso os argumentos e o resultado original do 'sits_coverage':

In [49]:
sits_coverage(service = "EOCUBES",
              name = "MOD13Q1/006",
              bands = "ndvi",
              geom = sf::read_sf(paste0(getwd(), "/geom/geom.shp")))

[1] "EOCUBES"
[1] "MOD13Q1/006"
[1] "ndvi"
Simple feature collection with 1 feature and 1 field
geometry type:  POLYGON
dimension:      XY
bbox:           xmin: -6209251 ymin: -1270455 xmax: -6138769 ymax: -1145154
epsg (SRID):    NA
proj4string:    +proj=sinu +lon_0=0 +x_0=0 +y_0=0 +a=6371007.181 +b=6371007.181 +units=m +no_defs
# A tibble: 1 x 2
    FID                                                                 geometry
  <dbl>                                                            <POLYGON [m]>
1    27 ((-6209251 -1145154, -6142685 -1146623, -6138769 -1269966, -6207293 -12…


r_objs                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

Esse exemplo demonstra o que o sits.rep faz internamente. Nem todas as funções do sits são sobrescritas, depende da necessidade de armazenar um dado específico. 
Os parâmetro da função sobrescrita não são modificados, para que o resultado final possa ser o mesmo da sua contraparte original. 

## Reprodução dos Resultados

Após finalizado a execução do experimento, é possível criar um script reprodutível desse experimento para ser executado em um container do Docker. O sits.rep disponibiliza a função 'sits.rep_reproduce' que irá criar scripts, metadados, Dockerfile e copiar o arquivo de hash para reproduzir e verificar os resultados. O arquivo de Dockerfile irá conter todas as informações de dependências de sistema operacional e de pacotes de linguagem de programação, para um container possa ser criado simulando o ambiente original. Na imagem a seguir é demonstrado a estrutura do diretório após a execução da função 'sits.rep_reproduce'.


![alt text](img/rep-docker.png "Title")




Para executar o 'sits_rep.reproduce', é necessário informar o nome do experimento, juntamente com a sua árvore, e o nome do diretório aonde será armazenado o script e os dados para a reprodução. Logo abaixo, é demonstrado o uso da função 'sits.rep_reproduce':


In [ ]:
sits.rep::sits.rep_reproduce("arv_1", "classification", "rep_deeplearnings")

Finalizado a execução, o pesquisador poderá carregar o diretório 'rep_deeplearning' em uma base de dados aberta para acompanhar um texto científico em suas publicações. Para reproduzir o resultado, basta executar o comando 'docker build' em uma máquina que possua a tecnologia Docker instalado. Esta tecnologia permite virtualizar ambientes computacionais controlados e executar scripts. O ambiente computacional é criado através dos dados de contexto obtidos durante a execução dos scripts que se materializou em um Dockerfile. Por tanto, são instaladas todas as dependências de sistema operacional, pacotes, com as suas respectivas versões no momento da execução original, além de configurar a semente do gerador de números aleatórios e outras informações de sistema.  Nesta máquina virtual, o script reprodutível é executado e é criado um arquivo hash reprodutível dos resultados. No final do processo, o arquivo hash original é comparado com o arquivo reprodutível, para verificar se os resultados produzidos são exatamente os mesmos do experimento original.

No comando abaixo é criado uma imagem chamado 'rep_in_jupyter' que irá executar o script reprodutível automaticamente:

In [7]:
base::system("docker build -t rep_in_jupyter /home/rafael/sits.rep/arv_1/reproducible/rep_deeplearnings")

O container irá instalar as depêndencias do pacote, executar o script reprodutível e verificar o resultado. Será imprimido no terminal se o resultado foi ou não reprodutível.

Referencias:
\cite{doi:10.1198/106186007X178663, 10.7717/peerj.148, Boettiger:2015:IDR:2723872.2723882, doi:10.1111/tgis.12195}
et al \cite{gertler2018make} 
\cite{doi:10.1029/2012EO160003, doi:10.1002/2015EA000136}
https://www.okhub.org/ (propósito geral)

https://github.com/geosec/knowledge_hub
http://www.esensing.org/
nust2017opening
https://www.earthobservations.org/article.php?id=366
artigo_alber

\cite{10.1371/journal.pmed.0020124, prinz2011believe, baker20161}
\cite{reich2006genepattern, greff2017sacred, nust2017opening, govoni2019qresp}